In [84]:
import torch
import torch.nn as nn
import sklearn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [85]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


In [86]:
data = pd.read_csv("/content/fashion-mnist_train.csv")
data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
data = data.fillna(0)

In [88]:
x = data.drop("label", axis = 1).to_numpy()
y = data['label'].to_numpy()

In [89]:
x = x/255.0
y = y/255.0

In [90]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [91]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype = torch.float32)
    self.labels = torch.tensor(labels, dtype = torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]



In [92]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [93]:
train_dataloader  = DataLoader(train_dataset, batch_size = 128, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size = 128, shuffle=True, pin_memory=True)

In [94]:
class mynn(nn.Module):
  def __init__(self, num_dimension):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_dimension, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(64, 10),
    )

  def forward(self, x):
    return self.model(x)

In [95]:
model = mynn(X_train.shape[1])
model.to(device)

mynn(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [96]:
learning_rate = 0.01
epochs = 50

In [97]:
optimizer = optim.SGD(model.parameters(), lr = learning_rate, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [98]:
for epoch in range(epochs):
  total_epoch_loss = 0
  for batch_features, batch_labels in train_dataloader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)
    outputs = model(batch_features)
    loss = criterion(outputs, batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_epoch_loss += loss.item()
  print(f"Epoch : {epoch + 1}, Loss : {total_epoch_loss/len(train_dataloader)}")

Epoch : 1, Loss : 0.19459623507161936
Epoch : 2, Loss : 0.01895322065303723
Epoch : 3, Loss : 0.010446756025155384
Epoch : 4, Loss : 0.007127354192237059
Epoch : 5, Loss : 0.005422310295825203
Epoch : 6, Loss : 0.004378767297292749
Epoch : 7, Loss : 0.0036916698589921
Epoch : 8, Loss : 0.003171806369597713
Epoch : 9, Loss : 0.002775094509124756
Epoch : 10, Loss : 0.002453852296806872
Epoch : 11, Loss : 0.0022357139165202775
Epoch : 12, Loss : 0.0020177610544487834
Epoch : 13, Loss : 0.0018672741760189335
Epoch : 14, Loss : 0.001716673423536122
Epoch : 15, Loss : 0.0015758520825766027
Epoch : 16, Loss : 0.001483365946294119
Epoch : 17, Loss : 0.0013760020358798404
Epoch : 18, Loss : 0.0013080028793774545
Epoch : 19, Loss : 0.0012578705858128766
Epoch : 20, Loss : 0.001191600273673733
Epoch : 21, Loss : 0.001122949494359394
Epoch : 22, Loss : 0.0010395943564362824
Epoch : 23, Loss : 0.0010511238522206744
Epoch : 24, Loss : 0.0009911331486267348
Epoch : 25, Loss : 0.0009502789905915657
Ep